# Plugin_iot_connection.py

This module simulates the IoT sensor connection plugin.
It captures an image of the e-waste and sends it to the backend server.


In [ ]:
import time
import socket
import json
import base64
import cv2
import numpy as np

def capture_image():
    # Simulate capturing an image (replace with actual camera capture code)
    image = np.random.randint(0, 255, (32, 32, 3), dtype=np.uint8)
    _, buffer = cv2.imencode('.jpg', image)
    return base64.b64encode(buffer).decode('utf-8')

def send_data_to_backend(image_data, host='127.0.0.1', port=5001):
    try:
        payload = {"image": image_data}
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.connect((host, port))
            s.sendall(json.dumps(payload).encode('utf-8'))
            print("[IoT] Image data sent to backend.")
    except Exception as e:
        print("[IoT] Failed to send image data:", e)

if __name__ == '__main__':
    while True:
        image_data = capture_image()
        send_data_to_backend(image_data)
        time.sleep(5)  # Send image every 5 seconds

        # Note: In a real-world scenario, you would handle exceptions and retries more robustly.


# Backend_server.py

This module handles incoming image data from IoT sensors and classifies it using a CNN model.


In [ ]:
import socket
import json
import base64
import cv2
import numpy as np
import time
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.imagenet_utils import preprocess_input

# Load the trained CNN model
model = load_model('e_waste_cnn_classifier.h5')

# List of category labels
categories = ['Computers', 'Mobiles', 'Batteries', 'Printers', 'Monitors',
              'TVs', 'Cables', 'Scanners', 'Servers', 'Others']

def classify_image(img):
    try:
        resized = cv2.resize(img, (32, 32))
        normalized = resized.astype('float32') / 255.0
        input_data = np.expand_dims(normalized, axis=0)
        predictions = model.predict(input_data)
        predicted_class = np.argmax(predictions)
        return categories[predicted_class], float(np.max(predictions))
    except Exception as e:
        print("[Backend] Classification error:", e)
        return "Unknown", 0.0

def start_backend_server(host='127.0.0.1', port=5001):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind((host, port))
        s.listen()
        print(f"[Backend] Server listening on {host}:{port}...")

        while True:
            conn, addr = s.accept()
            with conn:
                print(f"[Backend] Connection from {addr}")
                data = conn.recv(65536)
                if data:
                    try:
                        payload = json.loads(data.decode('utf-8'))
                        print("[Backend] Received Image Data")

                        # Decode image from base64 and save
                        image_bytes = base64.b64decode(payload['image'])
                        nparr = np.frombuffer(image_bytes, np.uint8)
                        img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
                        timestamp = int(time.time())
                        filename = f"e_waste_image_{timestamp}.jpg"
                        cv2.imwrite(filename, img)
                        print(f"[Backend] Image saved as {filename}")

                        # Classify the image
                        label, confidence = classify_image(img)
                        print(f"[Backend] Predicted Category: {label} ({confidence:.2f} confidence)")

                    except json.JSONDecodeError:
                        print("[Backend] Invalid JSON received")
                    except KeyError:
                        print("[Backend] Missing expected keys in payload")

if __name__ == '__main__':
    start_backend_server()